In [17]:
import os
import pandas as pd
import numpy as np

In [18]:
files = os.listdir("./data")

dates = ["2024-01", "2024-02", "2024-03", "2024-04"]

kline_1m = list()
funding_rate = list()
for date in dates:
    kline_1m.append(pd.read_csv(f"./data/BTCUSDT-1m-{date}.csv"))
    funding_rate.append(pd.read_csv(f"./data/BTCUSDT-fundingRate-{date}.csv"))
kline_1m = pd.concat(kline_1m, ignore_index=True)
funding_rate = pd.concat(funding_rate, ignore_index=True)

rate_map = funding_rate.set_index('calc_time')['last_funding_rate'].to_dict()
kline_1m['last_funding_rate'] = kline_1m['open_time'].map(lambda x: rate_map.get(x, 0))
print(kline_1m)


            open_time     open     high      low    close   volume  \
0       1704067200000  42314.0  42335.8  42289.6  42331.9  289.641   
1       1704067260000  42331.9  42353.1  42331.8  42350.4  202.444   
2       1704067320000  42350.4  42370.8  42349.6  42360.2  271.521   
3       1704067380000  42360.1  42405.8  42360.1  42405.8  392.238   
4       1704067440000  42405.7  42437.2  42405.7  42437.1  568.366   
...               ...      ...      ...      ...      ...      ...   
174235  1714521300000  60731.7  60768.5  60727.1  60756.5   53.951   
174236  1714521360000  60756.5  60759.1  60734.5  60746.4   63.081   
174237  1714521420000  60746.3  60754.1  60667.3  60679.1  214.419   
174238  1714521480000  60679.0  60685.2  60615.7  60615.8  155.238   
174239  1714521540000  60615.7  60657.9  60610.7  60651.2  142.134   

           close_time  quote_volume  count  taker_buy_volume  \
0       1704067259999  1.225616e+07   3310           175.211   
1       1704067319999  8.572241

In [19]:
print(kline_1m.columns)
"""
open_time	            K线图开盘时间(unix格式)
open	                开盘价
high	                最高价
low	                    最低价
close                   收盘价
volume                  成交量
close_time	            K线图收盘时间(unix格式)
quote_volume	        报价币成交量
count	                成单数
taker_buy_volume	    在此期间吃单方买入的基础币数量
taker_buy_quote_volume	在此期间吃单方买入的报价币数量
ignore	                忽略
last_funding_rate       资金费率
"""

kline_1m = kline_1m.drop(["taker_buy_volume", "close_time", "ignore"], axis=1)
print(kline_1m.columns)

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume',
       'ignore', 'last_funding_rate'],
      dtype='object')
Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume',
       'count', 'taker_buy_quote_volume', 'last_funding_rate'],
      dtype='object')


In [20]:
# 计算红绿柱 红1 绿0
kline_1m['color'] = kline_1m.apply(lambda row: 1 if row['close'] > row['open'] else 0, axis = 1)

# 计算价格范围  
kline_1m['price_range'] = kline_1m['high'] - kline_1m['low']  
  
# 计算实体长度（阳线为close-open，阴线为open-close）  
# 使用np.where进行条件判断  
kline_body = np.where(kline_1m['close'] > kline_1m['open'],   
                      kline_1m['close'] - kline_1m['open'],   
                      kline_1m['open'] - kline_1m['close'])  
  
# 计算上影线长度（阳线为high-close，阴线为high-open）  
kline_upper_shadow = np.where(kline_1m['close'] > kline_1m['open'],   
                              kline_1m['high'] - kline_1m['close'],   
                              kline_1m['high'] - kline_1m['open'])  
  
# 计算下影线长度（阳线为open-low，阴线为close-low）  
kline_lower_shadow = np.where(kline_1m['close'] > kline_1m['open'],   
                              kline_1m['open'] - kline_1m['low'],   
                              kline_1m['close'] - kline_1m['low'])  
  
# 计算实体比例、上影线比例和下影线比例  
kline_1m['kline_body_ratio'] = kline_body / kline_1m['price_range']  
kline_1m['kline_upper_shadow_ratio'] = kline_upper_shadow / kline_1m['price_range']  
kline_1m['kline_lower_shadow_ratio'] = kline_lower_shadow / kline_1m['price_range']

# 下个Kline
kline_1m['signal'] = (kline_1m['close'].shift(-1) - kline_1m['open'].shift(-1)) / kline_1m["close"]

# 由于rolling是向前滚动的，我们先反转DataFrame  
kline_1m_reversed = kline_1m[::-1]
signal_5_reversed = kline_1m_reversed['signal'].rolling(window=5).sum()
signal_5 = signal_5_reversed[::-1]
kline_1m['signal_5'] = signal_5

# 保存文件
kline_1m.to_csv("BTCUSDT_BINANCE_1m_2024_01_04.csv", index=False)
print(kline_1m.columns)
# print(kline_1m.tail(50))

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume',
       'count', 'taker_buy_quote_volume', 'last_funding_rate', 'color',
       'price_range', 'kline_body_ratio', 'kline_upper_shadow_ratio',
       'kline_lower_shadow_ratio', 'signal', 'signal_5'],
      dtype='object')
